In [1]:
import pandas as pd
import pathlib

## Calificar examen estudiantes: Dimensión 7

In [2]:
ruta_git = pathlib.Path().cwd().parent.parent.parent

In [3]:
est = pd.read_excel(ruta_git/'data/descargables/EstudiantesCFK.xlsx')

print(est.shape)
est.columns

(20750, 74)


Index(['N registro', 'Deseo participar en el estudio', 'Código IE', 'Grupo',
       'Nombre', 'Fecha', 'ID', 'Número de lista', 'Edad', 'Sexo',
       'Sector vivienda', 'Internet', 'Uso del dispositivo móvil',
       'Nivel escolaridad madre', 'Nivel escolaridad padre', 'Ocupación madre',
       'Ocupación padre', '¿Con quién vives?', 'Grado', '1.1. Ingeniería',
       '1.2 Matemáticas', '1.3 Educación', '1.4 Medicina',
       '1.5 Diseño gráfico', '1.6 Química', '1.7 Enfermería',
       '1.8 Desarrollo de software',
       '2.1 Soy capaz de sacar buenas notas en esta asignatura',
       '2.2 Si me va bien en esta asignatura, me ayudará en mi futura ocupación',
       '2.3 A mis padres les gustaría que eligiera un futuro profesional relacionado a esta asignatura',
       '2.4 Sé de alguien en mi familia que utiliza conocimientos relacionados a esta asignatura en su ocupación',
       'Comentarios 1-2', 'Un algoritmo es:',
       '¿Para qué sirven los algoritmos?', 'Un bucle es:',
    

##### Pasar respuestas a números

In [4]:
respuestas_correctas = {
    "Un algoritmo es:":"Una secuencia lógica de pasos para realizar una tarea",
    "¿Para qué sirven los algoritmos?":"Para planificar la solución a un problema",
    "Un bucle es:":"Un conjunto de instrucciones que se ejecuta mientras se cumpla una condición",
    "¿Cuál de las siguientes opciones sí le permite al robot completar la misión de fotografiar cada tortuga?":"a.",
    "¿Qué mensaje deseaba enviar la líder Wayuú?":"c. Nublado",
    "¿Cuál de los siguientes códigos permite que el robot complete su misión sembrando café?":"a.",
    "¿Cuál será la foto con más vistas?":"c) Julio",
    "Ayuda al robot verde a salir del laberinto":"b.",
    "Óscar lleva 2 loncheras a la escuela todos los días ¿Cuál de las siguientes afirmaciones es falsa?":"c) Si Óscar empaca Deditos para merendar, puede hacer Arroz de pollo para almorzar",
    "¿Cuál de las siguientes hamburguesas tiene los ingredientes A, E y F?":"a.",
    "¿Qué botella debe cambiarse de color para que el resultado final sea una botella de color blanco?":"a) La botella B debe ser verde"}

de_acuerdo = {"Totalmente en desacuerdo":1,"En desacuerdo":2, "Neutro":3,
"De acuerdo":4,"Totalmente de acuerdo":5}

interesa_carreras = {"No la conozco":0,"La evitaría":1,"Me interesa poco":2,"Está entre mis preferidas":3}

estereotipo_h = {'a) Seguramente un hombre':2, 'b) Quizás un hombre':1,
                 'c) Creo que puede ser un hombre o una mujer':0,
                 'd) Quizás una mujer': -1, 'e) Seguramente una mujer':-2}

estereotipo_m = {'a) Seguramente un hombre':-2, 'b) Quizás un hombre':-1,
                 'c) Creo que puede ser un hombre o una mujer':0,
                 'd) Quizás una mujer': 1, 'e) Seguramente una mujer':2}
#The scores range from -2= ‘counter- stereotypic answer’ to 2= ‘stereotype congruent answer’ for all items.
genero_estereotipos = {'5.1 ¿Quién crees que ganará el concurso de matemáticas?':estereotipo_h,
                       '5.2 ¿Quién crees que es capitán del barco?':estereotipo_h,
                       '5.3 ¿Quién es la persona excluida de la construcción de la casa de madera?':estereotipo_m,
                       '5.4 ¿Quién crees que es el personaje que está sentado y esperando junto a la ventana?':estereotipo_m,
                       '5.5 ¿Quién es la persona que trabaja en educación?':estereotipo_m,
                       '5.6 ¿Quién crees que es la persona que prefiere estudiar ingeniería?':estereotipo_h}


In [5]:
col_index = ['Edad', 'Sexo', 'Código IE', '¿Te reconoces como una persona con algún tipo de discapacidad?']
col_caract = ['N registro','Edad', 'Sexo', 'Sector vivienda', 'Internet', 'Uso del dispositivo móvil',
              'Nivel escolaridad madre', 'Nivel escolaridad padre', 'Ocupación madre',
              'Ocupación padre', '¿Con quién vives?', 'Grado', 'Código IE',
              'Grupo', 'Conoce GreenTIC', 'Número de lista', '¿Te reconoces como una persona con algún tipo de discapacidad?']
#col_conocePC = est.filter(regex=r'^3.*', axis=1).columns.tolist()[0]
#col_PC = est.filter(regex=r'^3.*', axis=1).columns.tolist()[1:5]
col_autoeficacia = est.filter(regex=r'^3.*', axis=1).columns.tolist()
col_carreras = est.filter(regex=r'^1.', axis=1).columns.tolist()
col_interes = est.filter(regex=r'^2.1|2.2', axis=1).columns.tolist()
col_conocimiento = list(respuestas_correctas.keys())
col_ambiental = est.filter(regex=r'^4.', axis=1).columns.tolist()
col_genero = est.filter(regex=r'^5.', axis=1).columns.tolist()
col_genero

['5.1 ¿Quién crees que ganará el concurso de matemáticas?',
 '5.2 ¿Quién crees que es capitán del barco?',
 '5.3 ¿Quién es la persona excluida de la construcción de la casa de madera?',
 '5.4 ¿Quién crees que es el personaje que está sentado y esperando junto a la ventana?',
 '5.5 ¿Quién es la persona que trabaja en educación?',
 '5.6 ¿Quién crees que es la persona que prefiere estudiar ingeniería?']

In [6]:
df_conocimientos = est[['N registro']+col_conocimiento].melt(id_vars='N registro', value_name='Respuesta estudiante', var_name='Pregunta')
df_conocimientos['Respuesta correcta'] = df_conocimientos['Pregunta'].replace(respuestas_correctas)
df_conocimientos['Puntaje conocimiento'] = 1*(df_conocimientos['Respuesta estudiante'] == df_conocimientos['Respuesta correcta'])
df_estudiantes = df_conocimientos.pivot_table(index='N registro', columns='Pregunta', values='Puntaje conocimiento').reset_index()



df_estudiantes= pd.merge(est[col_caract], df_estudiantes, on='N registro')
df_estudiantes

,N registro,Edad,Sexo,Sector vivienda,Internet,Uso del dispositivo móvil,Nivel escolaridad madre,Nivel escolaridad padre,Ocupación madre,Ocupación padre,...,Un algoritmo es:,Un bucle es:,"¿Cuál de las siguientes hamburguesas tiene los ingredientes A, E y F?",¿Cuál de las siguientes opciones sí le permite al robot completar la misión de fotografiar cada tortuga?,¿Cuál de los siguientes códigos permite que el robot complete su misión sembrando café?,¿Cuál será la foto con más vistas?,¿Para qué sirven los algoritmos?,¿Qué botella debe cambiarse de color para que el resultado final sea una botella de color blanco?,¿Qué mensaje deseaba enviar la líder Wayuú?,Óscar lleva 2 loncheras a la escuela todos los días ¿Cuál de las siguientes afirmaciones es falsa?
0,1,14.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Técnico o tecnólogo,Titulo universitario,Trabaja,Trabaja,...,0,0,1,1,1,0,0,1,0,1
1,2,16.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Trabaja,Trabaja,...,0,1,0,0,1,0,0,0,0,0
2,3,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Trabaja,Trabaja,...,1,0,1,1,0,0,1,0,0,0
3,4,15.0,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,No sé,No sé,Trabaja,Trabaja,...,1,0,0,0,0,0,0,0,1,0
4,5,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Titulo universitario,Secundaría,Trabaja,Trabaja,...,1,0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20745,21389,11.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),No sé,No sé,Trabaja,Trabaja,...,0,0,0,0,0,0,0,0,0,0
20746,21390,12.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),Secundaría,No sé,Trabaja,Trabaja,...,0,0,1,0,0,0,0,0,0,0
20747,21391,13.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Secundaría,Secundaría,Trabaja,Trabaja,...,0,0,1,1,0,0,0,0,1,0
20748,21392,13.0,Masculino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Cuida el hogar,Trabaja,...,1,0,1,1,0,0,1,1,1,1


In [7]:
# carreras_df = est[['N registro'] + col_carreras].replace(interesa_carreras)
# df_estudiantes = pd.merge(df_estudiantes, carreras_df, on='N registro')
#
# genero_df = est[['N registro']+col_genero].replace(genero_estereotipos)
# df_estudiantes = pd.merge(df_estudiantes, genero_df, on='N registro')

df_ambiente = est[['N registro']+col_ambiental].replace(de_acuerdo)

df_estudiantes = pd.merge(df_estudiantes, df_ambiente, on='N registro')

df_estudiantes = pd.merge(df_estudiantes, est[['N registro']+col_autoeficacia].astype(int), on='N registro')

df_estudiantes

,N registro,Edad,Sexo,Sector vivienda,Internet,Uso del dispositivo móvil,Nivel escolaridad madre,Nivel escolaridad padre,Ocupación madre,Ocupación padre,...,3.1 Siento que soy capaz de explicar lo que es el pensamiento computacional,3.2 Siento que puedo enumerar las sub-habilidades que componen el pensamiento computacional,3.3 Siento que soy capaz de dar ejemplos para explicar las sub-habilidades del pensamiento computacional,3.4 Siento que puedo explicar la forma en que las sub-habilidades del pensamiento computacional se correlacionan con la programación,3.5 Siento que puedo analizar un ejercicio y determinar qué sub-habilidades de pensamiento computacional busca desarrollar,3.6 Siento que puedo resolver problemas a través de programación,3.7 Siento que puedo implementar algoritmos,3.8 Siento que puedo crear un programa de computador,3.9 Siento que puedo automatizar tareas a través de la programación,3.10 Siento que puedo utilizar la computación para resolver problemas simples
0,1,14.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Técnico o tecnólogo,Titulo universitario,Trabaja,Trabaja,...,3,1,2,2,3,3,2,1,2,3
1,2,16.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Trabaja,Trabaja,...,5,4,5,5,5,5,5,5,5,5
2,3,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Trabaja,Trabaja,...,1,1,2,1,1,1,3,2,2,2
3,4,15.0,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,No sé,No sé,Trabaja,Trabaja,...,2,3,2,3,4,4,4,1,3,5
4,5,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Titulo universitario,Secundaría,Trabaja,Trabaja,...,3,3,3,3,3,3,4,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20745,21389,11.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),No sé,No sé,Trabaja,Trabaja,...,5,4,1,1,2,4,2,5,1,4
20746,21390,12.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),Secundaría,No sé,Trabaja,Trabaja,...,5,3,4,3,5,4,2,4,5,5
20747,21391,13.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Secundaría,Secundaría,Trabaja,Trabaja,...,1,1,1,1,1,1,1,2,2,3
20748,21392,13.0,Masculino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Cuida el hogar,Trabaja,...,4,4,4,4,5,5,4,5,4,5


#### Calcular escalas

In [8]:
escalas_dict = {'medioambiente':
                    {'cols':list(est.filter(regex='^4.*').columns),
                     'cargas':[0.620,0.648,0.732,0.638,0.705,0.707,0.670]},
                'autoeficaciaPC':
                    {'cols':list(est.filter(regex='^3.1 |3.[2-5]|^3.7').columns),
                'cargas':[0.724,0.822,0.782,0.745,0.574,0.390]},
                'autoeficaciaProg':{
                    'cols':list(est.filter(regex='^3.6 |3.[8-9]|3.10').columns),
                    'cargas':[0.637,0.490,0.755,0.753]
                }}

In [9]:
for k in list(escalas_dict.keys()):
    df_estudiantes[k] = 100*df_estudiantes[escalas_dict[k]['cols']].add(-1).multiply(escalas_dict[k]['cargas']).sum(axis=1)/(4*sum(escalas_dict[k]['cargas']))

In [10]:
resultados = df_estudiantes[col_conocimiento].sum(axis=1)
media = resultados.mean()
desv = resultados.std()

df_estudiantes['conocimiento'] = 50+(10*(resultados - media)/desv)



In [11]:
df_estudiantes

,N registro,Edad,Sexo,Sector vivienda,Internet,Uso del dispositivo móvil,Nivel escolaridad madre,Nivel escolaridad padre,Ocupación madre,Ocupación padre,...,3.5 Siento que puedo analizar un ejercicio y determinar qué sub-habilidades de pensamiento computacional busca desarrollar,3.6 Siento que puedo resolver problemas a través de programación,3.7 Siento que puedo implementar algoritmos,3.8 Siento que puedo crear un programa de computador,3.9 Siento que puedo automatizar tareas a través de la programación,3.10 Siento que puedo utilizar la computación para resolver problemas simples,medioambiente,autoeficaciaPC,autoeficaciaProg,conocimiento
0,1,14.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Técnico o tecnólogo,Titulo universitario,Trabaja,Trabaja,...,3,3,2,1,2,3,57.621822,27.947733,33.538899,57.920200
1,2,16.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Trabaja,Trabaja,...,5,5,5,5,5,5,75.000000,94.909586,100.000000,43.662878
2,3,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Trabaja,Trabaja,...,1,1,3,2,2,2,79.713983,9.673025,18.956357,53.167759
3,4,15.0,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,No sé,No sé,Trabaja,Trabaja,...,4,4,4,1,3,5,92.521186,46.643547,61.034156,43.662878
4,5,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Titulo universitario,Secundaría,Trabaja,Trabaja,...,3,3,4,3,3,3,71.461864,52.415160,50.000000,53.167759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20745,21389,11.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),No sé,No sé,Trabaja,Trabaja,...,2,4,2,5,1,4,93.135593,39.175130,58.159393,34.157996
20746,21390,12.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),Secundaría,No sé,Trabaja,Trabaja,...,5,4,2,4,5,5,85.709746,68.503839,89.307400,43.662878
20747,21391,13.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Secundaría,Secundaría,Trabaja,Trabaja,...,1,1,1,2,2,3,100.000000,0.000000,26.100569,48.415319
20748,21392,13.0,Masculino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Cuida el hogar,Trabaja,...,5,5,4,5,4,5,89.592161,78.554620,92.836812,67.425082


In [12]:
estudiantes = pd.merge(est, df_estudiantes[['N registro', 'conocimiento', 'autoeficaciaPC', 'autoeficaciaProg', 'medioambiente']])

In [13]:
df_interes = est[['N registro']+col_interes].melt(id_vars='N registro', value_name='Seleccion', var_name='Pregunta')
df_interes['Interes en tecnologia'] = 1*(df_interes['Seleccion'].str.contains('Tecnología'))
df_puntaje_interes = df_interes.pivot_table(index='N registro', values=['Interes en tecnologia'], aggfunc=['sum', 'max']).reset_index()
df_puntaje_interes.columns = ['N registro', 'Puntaje interes', 'Interesado en tecnologia']
df_puntaje_interes['Puntaje interes'] = 100*(df_puntaje_interes['Puntaje interes']/2)
#df_puntaje_interes = (100*df_interes.groupby('index')['Interes en tecnologia'].sum()/4).reset_index()
df_puntaje_interes

,N registro,Puntaje interes,Interesado en tecnologia
0,1,100.0,1
1,2,50.0,1
2,3,50.0,1
3,4,100.0,1
4,5,0.0,0
...,...,...,...
20745,21389,0.0,0
20746,21390,0.0,0
20747,21391,0.0,0
20748,21392,100.0,1


In [14]:
df_7 = pd.merge(estudiantes, df_puntaje_interes, on='N registro')
df_7

,N registro,Deseo participar en el estudio,Código IE,Grupo,Nombre,Fecha,ID,Número de lista,Edad,Sexo,...,Tipo de discapacidad,¿Te reconoces como una persona con algún tipo de discapacidad?,Conoce GreenTIC,Instrumento,conocimiento,autoeficaciaPC,autoeficaciaProg,medioambiente,Puntaje interes,Interesado en tecnologia
0,1,Sí,166,2,NaN,19/04,166090203,3,14.0,Masculino,...,NaN,No,No,Encuesta estudiantes,57.920200,27.947733,33.538899,57.621822,100.0,1
1,2,Sí,166,2,NaN,19/04,166090207,7,16.0,Masculino,...,NaN,No,No,Encuesta estudiantes,43.662878,94.909586,100.000000,75.000000,50.0,1
2,3,Sí,166,2,NaN,19/04,166090230,30,14.0,Femenino,...,NaN,No,No,Encuesta estudiantes,53.167759,9.673025,18.956357,79.713983,50.0,1
3,4,Sí,166,2,NaN,19/04,166090206,6,15.0,Masculino,...,NaN,No,No,Encuesta estudiantes,43.662878,46.643547,61.034156,92.521186,100.0,1
4,5,Sí,166,2,NaN,19/04,166090210,10,14.0,Femenino,...,NaN,No,No,Encuesta estudiantes,53.167759,52.415160,50.000000,71.461864,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20745,21389,Sí,33,2,Sara Sofia Vargas Avila,06/06,33060241,41,11.0,Femenino,...,NaN,No,No,Encuesta estudiantes,34.157996,39.175130,58.159393,93.135593,0.0,0
20746,21390,Sí,33,2,Deivid Forero Lemus,06/06,33060208,8,12.0,Femenino,...,NaN,No,No,Encuesta estudiantes,43.662878,68.503839,89.307400,85.709746,0.0,0
20747,21391,Sí,33,2,Alejandro Padilla Rios,06/06,33060225,25,13.0,Masculino,...,NaN,No,No,Encuesta estudiantes,48.415319,0.000000,26.100569,100.000000,0.0,0
20748,21392,Sí,125,1,Camilo Andrés Herrera Corcho,06/06,125090121,21,13.0,Masculino,...,NaN,No,Sí,Encuesta estudiantes,67.425082,78.554620,92.836812,89.592161,100.0,1


In [15]:
df_7['autoeficaciaProg'].mean()

61.9480702315905

In [16]:
df_7['Media autoeficaciaProg'] = df_7['autoeficaciaProg'].mean()
df_7['Std autoeficaciaProg'] = df_7['autoeficaciaProg'].std()
df_7['Media autoeficaciaPC'] = df_7['autoeficaciaPC'].mean()
df_7['Std autoeficaciaPC'] = df_7['autoeficaciaPC'].std()

In [17]:
df_7['Media nacional interés'] = df_7['Puntaje interes'].mean()
df_7['Std interes'] = df_7['Puntaje interes'].std()

### Calificar Dimensión

In [18]:
cols_dimension7 = ['conocimiento', 'Media autoeficaciaProg','Puntaje interes', 'Interesado en tecnologia', 'Media autoeficaciaPC', 'autoeficaciaPC', 'autoeficaciaProg', 'Media nacional interés','Std interes','Std autoeficaciaPC','Std autoeficaciaProg', 'N registro']
agrupadores_dimension7 = ['mean', 'max', 'mean', 'sum', 'max', 'mean','mean', 'max','max','max','max', 'count']
agg_funcs = dict(zip(cols_dimension7, agrupadores_dimension7))

In [19]:
agg_funcs

{'conocimiento': 'mean',
 'Media autoeficaciaProg': 'max',
 'Puntaje interes': 'mean',
 'Interesado en tecnologia': 'sum',
 'Media autoeficaciaPC': 'max',
 'autoeficaciaPC': 'mean',
 'autoeficaciaProg': 'mean',
 'Media nacional interés': 'max',
 'Std interes': 'max',
 'Std autoeficaciaPC': 'max',
 'Std autoeficaciaProg': 'max',
 'N registro': 'count'}

In [20]:
df_institucion = df_7.pivot_table(index=['Código IE'], values=cols_dimension7, aggfunc=agg_funcs).rename(columns={'N registro':'Cantidad estudiantes'})
df_institucion['% Interesado'] = (df_institucion['Interesado en tecnologia']/df_institucion['Cantidad estudiantes'])*100

In [21]:
## Cuántas desviaciones estándar respecto a la media
df_institucion['Diff autoeficaciaPC'] = (df_institucion['autoeficaciaPC'] - df_institucion['Media autoeficaciaPC'])/df_institucion['Std autoeficaciaPC']

df_institucion['Diff autoeficaciaProg'] = (df_institucion['autoeficaciaProg'] - df_institucion['Media autoeficaciaProg'])/df_institucion['Std autoeficaciaProg']

df_institucion['Diferencia interes'] = (df_institucion['Puntaje interes'] - df_institucion['Media nacional interés'])/df_institucion['Std interes']

In [22]:
df_institucion['Media autoeficaciaProg']

Código IE
1      61.94807
2      61.94807
3      61.94807
4      61.94807
5      61.94807
         ...   
248    61.94807
249    61.94807
250    61.94807
251    61.94807
252    61.94807
Name: Media autoeficaciaProg, Length: 234, dtype: float64

In [23]:
df_institucion['Nivel autoeficaciaPC'] =  pd.cut(df_institucion['autoeficaciaPC'], bins=[0, 20, 30, 50, 60, 80,100], labels=['1A', '1B', '2A', '2B','3', '5'], include_lowest=True)

df_institucion['Nivel autoeficaciaProg'] =  pd.cut(df_institucion['autoeficaciaProg'], bins=[0, 20, 30, 50, 60, 100], labels=['1A', '1B', '2A', '2B', '5'], include_lowest=True)

df_institucion['Nivel conocimiento'] =  pd.cut(df_institucion['conocimiento'], bins=[0, 30, 40, 50, 60, 100], labels=['1A', '1B', '2A', '2B', '5'], include_lowest=True)

df_institucion['Nivel Interes'] = pd.cut(df_institucion['% Interesado'], bins=[0, 20, 40, 60, 80, 100], labels=['1A', '1B', "2A", "2B", '5'], include_lowest=True)

df_institucion['Nivel Interes pares'] = pd.cut(df_institucion['Diferencia interes'], bins=[-3,-1,0,1,2,3], labels=['1B', "2A", "3", '4','5'], include_lowest=True)

df_institucion['Nivel AutoeficaciaPC pares'] = pd.cut(df_institucion['Diff autoeficaciaPC'], bins=[-3, 0, 1.5, 2, 4], labels=['2A',"2B", "3", '4'], include_lowest=True)

df_institucion['Nivel AutoeficaciaProg pares'] = pd.cut(df_institucion['Diff autoeficaciaProg'], bins=[-3, 0, 1.5, 2, 4], labels=['2A',"2B", "3", '4'], include_lowest=True)


df_institucion

,Interesado en tecnologia,Media autoeficaciaPC,Media autoeficaciaProg,Media nacional interés,Cantidad estudiantes,Puntaje interes,Std autoeficaciaPC,Std autoeficaciaProg,Std interes,autoeficaciaPC,...,Diff autoeficaciaPC,Diff autoeficaciaProg,Diferencia interes,Nivel autoeficaciaPC,Nivel autoeficaciaProg,Nivel conocimiento,Nivel Interes,Nivel Interes pares,Nivel AutoeficaciaPC pares,Nivel AutoeficaciaProg pares
Código IE,,,,,,,,,,,,,,,,,,,,,
1,57,44.330723,61.94807,33.595181,119,35.294118,26.755191,27.277686,38.390682,45.524227,...,0.044608,0.158348,0.044254,2A,5,2A,2A,3,2B,2B
2,26,44.330723,61.94807,33.595181,38,51.315789,26.755191,27.277686,38.390682,40.168083,...,-0.155583,-0.414905,0.461586,2A,2B,2A,2B,3,2A,2A
3,32,44.330723,61.94807,33.595181,57,39.473684,26.755191,27.277686,38.390682,48.697900,...,0.163227,0.250608,0.153123,2A,5,2A,2A,3,2B,2B
4,30,44.330723,61.94807,33.595181,46,51.086957,26.755191,27.277686,38.390682,47.185948,...,0.106717,0.052241,0.455626,2A,5,2B,2B,3,2B,2B
5,77,44.330723,61.94807,33.595181,99,57.070707,26.755191,27.277686,38.390682,50.908265,...,0.245842,0.453339,0.611490,2B,5,2B,2B,3,2B,2B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,27,44.330723,61.94807,33.595181,58,29.310345,26.755191,27.277686,38.390682,33.630086,...,-0.399946,-0.263430,-0.111611,2A,2B,2A,2A,2A,2A,2A
249,50,44.330723,61.94807,33.595181,149,20.805369,26.755191,27.277686,38.390682,39.210915,...,-0.191358,-0.152764,-0.333149,2A,2B,2A,1B,2A,2A,2A
250,27,44.330723,61.94807,33.595181,83,20.481928,26.755191,27.277686,38.390682,40.542378,...,-0.141593,-0.131832,-0.341574,2A,2B,2A,1B,2A,2A,2A


In [26]:
df_institucion['Dimensión 7'] = df_institucion.filter(regex='^Nivel').min(axis=1)

In [27]:
df_dimension = df_institucion.filter(regex='^Nivel|Dim')
df_dimension.reset_index().to_excel('Dimensión 7.xlsx')

In [28]:
df_dimension

,Nivel autoeficaciaPC,Nivel autoeficaciaProg,Nivel conocimiento,Nivel Interes,Nivel Interes pares,Nivel AutoeficaciaPC pares,Nivel AutoeficaciaProg pares,Dimensión 7
Código IE,,,,,,,,
1,2A,5,2A,2A,3,2B,2B,2A
2,2A,2B,2A,2B,3,2A,2A,2A
3,2A,5,2A,2A,3,2B,2B,2A
4,2A,5,2B,2B,3,2B,2B,2A
5,2B,5,2B,2B,3,2B,2B,2B
...,...,...,...,...,...,...,...,...
248,2A,2B,2A,2A,2A,2A,2A,2A
249,2A,2B,2A,1B,2A,2A,2A,1B
250,2A,2B,2A,1B,2A,2A,2A,1B
